In [2]:
# 강의 https://www.youtube.com/watch?v=tQUtBR3K1TI
#https://colab.research.google.com/drive/1S3jKF6Jofvl48fHUFqQwS-0iWMgl-pWl?usp=sharing#scrollTo=pWiMRQC1Kikl
!pip install -qchromadb tiktoken transformers sentence_transformers langchain pypdf


In [1]:
import tiktoken

tokenizer = tiktoken.get_encoding("cl100k_base")

def tiktoken_len(text):
    tokens = tokenizer.encode(text)
    return len(tokens)

In [2]:

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFLoader

import os.path        
print(os.path.abspath("../test-data/GR_07_ver1066.pdf"))

loader = PyPDFLoader("../test-data/GR_07_ver1066.pdf")
pages = loader.load_and_split()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50, length_function = tiktoken_len)
texts = text_splitter.split_documents(pages)

from langchain.embeddings import HuggingFaceEmbeddings

model_name = "jhgan/ko-sbert-nli"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

docsearch = Chroma.from_documents(texts, hf)

/Users/stlim/Works/knd/customAIService/test-data/GR_07_ver1066.pdf


/var/folders/50/24dk5fd12ld821qvqv34k85m0000gn/T/ipykernel_18387/740814055.py:19: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf = HuggingFaceEmbeddings(


In [3]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOllama
from langchain_ollama.llms import OllamaLLM

from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
# model_name = "EEVE-Korean-Q5_K_M"
model_name = "llama3.2-vision"
llm = ChatOllama(model=model_name
                 , temperature=0
                 , streaming=True
                 , callbacks=[StreamingStdOutCallbackHandler()]
                 )


/var/folders/50/24dk5fd12ld821qvqv34k85m0000gn/T/ipykernel_18387/476702568.py:8: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model=model_name


In [5]:


qa = RetrievalQA.from_chain_type(llm = llm,
                                 chain_type = "stuff",
                                 retriever = docsearch.as_retriever(
                                    search_type="mmr",
                                    search_kwargs={'k':3, 'fetch_k': 6}),
                                 return_source_documents = True)

# query = "테스트케이스로 만들어줘. 테스트 케이스별로 출력. 형식은 markdown 파일로 해줘. 언어는 한글로 해줘"
query = "한글로 요약해줘. 언어는 한글로 해줘"

result = qa(query)

Number of requested results 6 is greater than number of elements in index 2, updating n_results = 2


리스트를 위한 디자인 규칙을 요약하면 다음과 같습니다.

**데이터 목록**

* 여러 정보가 같이 표시될 때 각 정보마다 1 줄씩 표시
* 잘리는 경우 말줄임표 사용
* 최대 텍스트 표시 라인 수는 화면 정의에 따라 결정
* 번역 시 잘리거나 공간이 남지 않도록 텍스트를 조정하거나 리스트 가변 적용 필요

**메뉴/기능/설정 목록**

* 고정된 레이블 텍스트를 표시하므로, 리스트 항목 높이는 고정
* 항목을 탭할 때의 동작을 알 수 있도록 리스트 우측에 꺾쇠(진입), 스위치 버튼(토글), 설정값(옵션들 중 선택)을 제공하거나 표시
* 리스트 항목 높이가 고정된 상태에서 최대 2 라인까지 텍스트 표시 가능하며, 더 길어질 경우 의미 전달에 문제가 없다면 말줄임표 사용
* 리스트 우측 끝에 표시되는 설정값은 1 라인까지 텍스트 표시 가능하며, 더 길어질 경우 말줄임표 사용
* 필요한 경우 설명 텍스트를 포함할 수 있으며, 설명 텍스트는 번역 시 잘리거나 공간이 남지 않도록 해야 하며, 이런 현상이 있는 항목에는 가변 리스트 적용 가능

In [8]:
print(result)
print(type (result) )
print([  item.metadata for item in result['source_documents'] ])
print(result['result'])

{'query': '체크리스트로 만들어줘. 형식은 html 으로 해줘', 'result': '사용자님, 요청하신 대로 Jira Workflow Action Handler 플러그인에 대한 정보를 체크리스트 형식으로 정리해 드렸습니다:\n\n1. 설치 방법:\n   a. 권한: Jira Administrator\n   b. 시스템 메뉴로 이동 -> 관리 앱 -> 앱 업로드 -> "플러그인 파일" 선택 -> 업로드 및 설치\n   c. 새 앱으로 이동 -> "Workflow Action Handler" 검색 -> 설치 -> 라이선스 추가 또는 구매\n\n2. Jira Admin 설정:\n   a. 구성 옵션\n   b. 설정\n   c. 기타 운영\n\n3. 앱 가이드:\n   a. 화면 소개:\n      i. 검색창 + 결과: 액션 + 적용된 전환 보기: 상태 -> 상태\n      ii. 상태 처리 + 전환 추가 + 유형 추가 등\n      iii. 각 액션별 사용된 플러그인 이름과 개발사 정보\n\n   b. 검색 방법:\n      i. 기본 검색 형식: 액션 + 타입 + 상태 등으로 검색\n      ii. 액션 텍스트 검색\n      iii. 트리거 조건 검증자 후처리 함수\n      iv. 플러그인 이름 및 개발사\n\n   c. 검색 결과에 따른 동작:\n      i. 검색 결과 워크플로우 전환\n      ii. 액션 + 순서 + 타입 + 플러그인 정보 + 상태 전환 보기 + 속성 + 삭제 + 편집\n\n   d. 전체 Workflow 보기:\n      i. 마우스 오버: 전환 화면 아래 작은 워크플로우 이미지에 마우스를 올려놓으면 큰 이미지 생성\n\n   e. 편집/삭제:\n      i. 편집: 검색된 액션에 대해 \'편집\' 선택 시 "편집" 화면 표시\n      ii. 저장 후 새로고침, 드래프트 저장\n\n   f. 상태 생성: 버튼 위치는 전환 영역 상단\n      i. DoTo + Inprogress + done에 따라 